# Visualization: Trading Session

In [1]:
import pandas as pd
import numpy as np

import altair as alt
import seaborn as sns

### 1. Define parameters and Load model

In [13]:
from trading_bot.agent import Agent

model_name = 'model_t-dqn_GOOG_10' #'model_t-dqn_GOOG_10' #'model_dqn_GOOG_50' #'model_GOOG_50'
test_stock = 'D:/share/finam/data/SBER/hourly/SBER_2022.csv' #'data/SBER_2019.csv' #'data/GOOG_2019.csv'
window_size = 16
debug = True

agent = Agent(window_size, pretrained=True, model_name=model_name)

### 2. Load test data

In [14]:
# read csv into dataframe
df = pd.read_csv(test_stock,index_col=[0], parse_dates=True)
#df['Adj Close'] = df['Close']
if 'Adj Close' not in df.columns: 
    #df = df.rename(columns={'Close': 'Adj Close'})
    df['Adj Close'] = df['Close']
    #df.set_index('Date')
    test_stock = 'data/tmp.csv'
    df.to_csv(test_stock)
    df = pd.read_csv(test_stock)
# filter out the desired features
df = df[['Date', 'Adj Close']]
# rename feature column names
df = df.rename(columns={'Adj Close': 'actual', 'Date': 'date'})
# convert dates from object to DateTime type
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates

df.head()

,date,actual
0,2020-01-03 11:00:00,255.77
1,2020-01-03 12:00:00,257.01
2,2020-01-03 13:00:00,257.20
3,2020-01-03 14:00:00,256.45
4,2020-01-03 15:00:00,254.65


### 3. Running Eval

In [4]:
df.columns

Index(['date', 'actual'], dtype='object')

In [5]:
import logging
import coloredlogs

from trading_bot.utils import show_eval_result, switch_k_backend_device, get_stock_data
from trading_bot.methods import evaluate_model

coloredlogs.install(level='DEBUG')
switch_k_backend_device()

test_data = get_stock_data(test_stock)
initial_offset = test_data[1] - test_data[0]

test_result, history = evaluate_model(agent, test_data, window_size, debug=False)
show_eval_result(model_name, test_result, initial_offset)

2021-02-17 12:34:03 HOME-PC root[46352] DEBUG switching to TensorFlow for CPU
2021-02-17 12:36:49 HOME-PC root[46352] INFO model_t-dqn_GOOG_10: +$9219.04



### 4. Visualize

In [6]:
%matplotlib widget

2021-02-17 12:43:09 HOME-PC matplotlib.pyplot[46352] DEBUG Loaded backend module://ipympl.backend_nbagg version unknown.


In [7]:
def visualize(df, history, title="trading session"):
    # add history to dataframe
    position = [history[0][0]] + [x[0] for x in history]
    actions = ['HOLD'] + [x[1] for x in history]
    df['position'] = position
    df['action'] = actions
    
    # specify y-axis scale for stock prices
    scale = alt.Scale(domain=(min(min(df['actual']), min(df['position'])) - 50, max(max(df['actual']), max(df['position'])) + 50), clamp=True)
    
    # plot a line chart for stock positions
    actual = alt.Chart(df).mark_line(
        color='green',
        opacity=0.5
    ).encode(
        x='date:T',
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale)
    ).interactive(
        bind_y=False
    )
    
    # plot the BUY and SELL actions as points
    points = alt.Chart(df).transform_filter(
        alt.datum.action != 'HOLD'
    ).mark_point(
        filled=True
    ).encode(
        x=alt.X('date:T', axis=alt.Axis(title='Date')),
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale),
        color='action'
    ).interactive(bind_y=False)

    # merge the two charts
    chart = alt.layer(actual, points, title=title).properties(height=300, width=1000)
    
    return chart

In [7]:
chart = visualize(df, history, title=test_stock)
chart

alt.LayerChart(...)

In [ ]:

b = 0
c = 0
for d in history:
    if d[1] == 'SELL':
        c += 0 if c == 0 else -1
        b += 0 if c == 0 else -1*d[0]
    elif d[1] == 'BUY':
        c += 1
        b += d[0]
    print(d,b,c)

In [ ]:
stop = 0.2
print(df.iloc[0:1])

mask = ((df['actual'] < df['actual']-df['actual']*stop) ).shift(1).iloc[1:]
df.iloc[1:].loc[mask].iloc[0]

In [ ]:
df.rolling(24)